In [1]:
# Import Statement - modified according to needs
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.multicomp import MultiComparison

In [ ]:
# Import csv and delete all rows with null values.

df_movie_original = pd.read_csv('moviedata.csv')
df_movie = df_movie_original.dropna()